In [2]:
from variant_utils.get_gene_info import get_gene_info
from variant_utils.clinvar_utils import queryClinVarVCF
from variant_utils.gnomad_utils import queryGnomAD
from variant_utils.spliceAI_utils import querySpliceAI
import urllib.request
from pathlib import Path
import os

# Set Java 21 environment BEFORE any imports that might use Java
os.environ['JAVA_HOME'] = '/jet/home/barazand/NEWOCEAN/java/jdk-21.0.9'
os.environ['PATH'] = f"/jet/home/barazand/NEWOCEAN/java/jdk-21.0.9/bin:{os.environ.get('PATH', '')}"

# Verify it's set correctly
import subprocess
result = subprocess.run(['java', '-version'], capture_output=True, text=True)
print("Java version being used:")
print(result.stderr)  # java -version outputs to stderr

def test_get_gene_info():
    get_gene_info('SRY').to_json(".cache/SRY.json")

def test_queryClinVarVCF():
    brca1_info = get_gene_info("BRCA1")
    sry_info = get_gene_info("SRY")
    pik3ca_info = get_gene_info("HBB")
    # set destination to save/reload ClinVar
    cache_dir = Path(".cache")
    cache_dir.mkdir(exist_ok=True)
    # download ClinVar release (e.g., 2018-12-17) if file is not present
    clinvar_filepath = cache_dir / "clinvar_20181217.vcf.gz"
    idx_filepath = cache_dir / "clinvar_20181217.vcf.gz.tbi"
    if not clinvar_filepath.exists():
        urllib.request.urlretrieve(f"https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/weekly/clinvar_20181217.vcf.gz",str(clinvar_filepath))
    if not idx_filepath.exists():
        urllib.request.urlretrieve(f"https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/weekly/clinvar_20181217.vcf.gz.tbi",str(idx_filepath))

    # brca1_clinvar_variants = queryClinVarVCF(str(clinvar_filepath), brca1_info.CHROM, brca1_info.chr_start, brca1_info.chr_end, "external_tools.json",write_dir=".cache")
    # brca1_clinvar_variants.to_json(".cache/BRCA1_clinvar.json")

    # brca1_clinvar_variants = queryClinVarVCF(str(clinvar_filepath), sry_info.CHROM, sry_info.chr_start, sry_info.chr_end, "external_tools.json",write_dir=".cache")
    # brca1_clinvar_variants.to_json(".cache/SRY_clinvar.json")

    pik3ca_clinvar_variants = queryClinVarVCF(str(clinvar_filepath), pik3ca_info.CHROM, pik3ca_info.chr_start, pik3ca_info.chr_end, "external_tools.json",write_dir=".cache")
    pik3ca_clinvar_variants.to_json(".cache/HBB_clinvar.json")


def test_queryGnomAD():
    gnomAD_tst_out = queryGnomAD('GRCh38', 'Y', 0, 100000000,"HGNC:11311",'external_tools.json',write_dir=".cache")
    gnomAD_tst_out.to_json(".cache/gnomAD_test.json")


# try:
#     test_get_gene_info()
#     print("test_get_gene_info() passed successfully!")
# except Exception as e:
#     print(f"test_get_gene_info() failed. Error: {e}")

try:
    test_queryClinVarVCF()
    print("test_queryClinVarVCF() passed successfully!")
except Exception as e:
    print(f"test_queryClinVarVCF() failed. Error: {e}")

try:
    # test_queryGnomAD()
    sry_info = get_gene_info("SRY")
    brca1_info = get_gene_info("BRCA1")
    pik3ca_info = get_gene_info("HBB")
    # sry_gnomad_variants = queryGnomAD("GRCh38",sry_info.CHROM, sry_info.chr_start, sry_info.chr_end, sry_info.HGNC_ID,"external_tools.json", write_dir=".cache")
    # sry_gnomad_variants.to_json(".cache/gnomAD_test_SRY.json")

    # sry_gnomad_variants = queryGnomAD("GRCh38",pik3ca_info.CHROM, pik3ca_info.chr_start, pik3ca_info.chr_end, pik3ca_info.HGNC_ID,"external_tools.json", write_dir=".cache")
    # sry_gnomad_variants.to_json(".cache/gnomAD_test_HBB.json")
    print("test_queryGnomAD() passed successfully!")
except Exception as e:
    print(f"test_queryGnomAD() failed. Error: {e}")



Java version being used:
java version "21.0.9" 2025-10-21 LTS
Java(TM) SE Runtime Environment (build 21.0.9+7-LTS-338)
Java HotSpot(TM) 64-Bit Server VM (build 21.0.9+7-LTS-338, mixed mode, sharing)



Using GATK jar /ocean/projects/cis250266p/barazand/proteins/gnomAD/gatk-4.6.1.0/gatk-package-4.6.1.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /ocean/projects/cis250266p/barazand/proteins/gnomAD/gatk-4.6.1.0/gatk-package-4.6.1.0-local.jar SelectVariants -V .cache/clinvar_20181217.vcf.gz -L 11:5225464-5227071 --exclude-filtered --output .cache/ClinVar_selectvariants_chr11:5225464-5227071.vcf
13:01:46.538 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/ocean/projects/cis250266p/barazand/proteins/gnomAD/gatk-4.6.1.0/gatk-package-4.6.1.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
SLF4J(W): Class path contains multiple SLF4J providers.
SLF4J(W): Found provider [org.apache.logging.slf4j.SLF4JServiceProvider@935493d]
SLF4J(W): Found provider [ch.qos.logback.classic.spi.LogbackServiceProvider@9b367c8]
SLF4J(W): See

test_queryClinVarVCF() passed successfully!
test_queryGnomAD() passed successfully!


In [3]:
#!/usr/bin/env python3
"""
Comprehensive analysis script for ClinVar and gnomAD JSON outputs

Usage:
    python analyze_variant_data.py --clinvar path/to/clinvar.json --gnomad path/to/gnomad.json

Or use as a module:
    from analyze_variant_data import analyze_clinvar, analyze_gnomad, compare_datasets
"""

import pandas as pd
import json
from pathlib import Path
import argparse
from typing import Dict, Tuple, Optional
import sys

def load_json_data(filepath: str | Path) -> pd.DataFrame:
    """Load JSON file and convert to DataFrame"""
    filepath = Path(filepath)
    if not filepath.exists():
        raise FileNotFoundError(f"File not found: {filepath}")
    
    with open(filepath, 'r') as f:
        data = json.load(f)
    
    # Handle different JSON formats
    if isinstance(data, list):
        df = pd.DataFrame(data)
    elif isinstance(data, dict):
        # Check if this is a dict of arrays (your case)
        first_val = next(iter(data.values()))
        if isinstance(first_val, dict):
            # This is the problematic format - need to reconstruct
            # Find the maximum length across all columns
            max_length = max(len(v) for v in data.values())
            reconstructed = []
            for i in range(max_length):
                row = {}
                for col in data.keys():
                    # Use .get() with None default for missing indices
                    row[col] = data[col].get(str(i), None)
                reconstructed.append(row)
            df = pd.DataFrame(reconstructed)
        else:
            df = pd.DataFrame([data])
    else:
        raise ValueError(f"Unexpected JSON format in {filepath}")
    
    return df


def analyze_clinvar(clinvar_json_path: str | Path) -> Dict:
    """
    Comprehensive analysis of ClinVar data
    
    Returns:
        Dict containing analysis results and the DataFrame
    """
    print("\n" + "="*80)
    print("CLINVAR ANALYSIS")
    print("="*80)
    
    df = load_json_data(clinvar_json_path)
    
    analysis = {
        'dataframe': df,
        'total_variants': len(df),
        'columns': list(df.columns),
        'shape': df.shape
    }
    
    # Basic statistics
    print(f"\n📊 BASIC STATISTICS:")
    print(f"   Total variants: {len(df)}")
    print(f"   Columns: {df.shape[1]}")
    
    # Safe chromosome extraction
    if 'CHROM' in df.columns:
        try:
            # Try to get unique values, handling cases where CHROM might be dict or other types
            chrom_values = df['CHROM'].apply(lambda x: str(x) if not isinstance(x, (dict, list)) else str(x)).unique().tolist()
            print(f"   Chromosomes: {chrom_values}")
        except:
            print(f"   Chromosomes: [complex data type]")
    
    # Column summary
    print(f"\n📋 AVAILABLE COLUMNS ({len(df.columns)}):")
    for col in df.columns:
        print(f"   - {col}")
    
    # Clinical significance analysis
    if 'CLNSIG' in df.columns:
        print(f"\n🔬 CLINICAL SIGNIFICANCE BREAKDOWN:")
        clnsig_counts = df['CLNSIG'].value_counts()
        analysis['clinical_significance'] = clnsig_counts.to_dict()
        for sig, count in clnsig_counts.items():
            percentage = (count / len(df)) * 100
            print(f"   {sig}: {count} ({percentage:.1f}%)")
    
    # Review status analysis
    if 'CLNREVSTAT' in df.columns:
        print(f"\n⭐ REVIEW STATUS BREAKDOWN:")
        revstat_counts = df['CLNREVSTAT'].value_counts()
        analysis['review_status'] = revstat_counts.to_dict()
        for status, count in revstat_counts.items():
            print(f"   {status}: {count}")
    
    # Pathogenicity flags (if present)
    pathogenicity_flags = ['is_pathogenic', 'is_benign', 'is_conflicting', 'is_VUS']
    if any(flag in df.columns for flag in pathogenicity_flags):
        print(f"\n🚩 PATHOGENICITY FLAGS:")
        for flag in pathogenicity_flags:
            if flag in df.columns:
                count = df[flag].sum() if df[flag].dtype == bool else df[flag].astype(bool).sum()
                percentage = (count / len(df)) * 100
                analysis[flag] = int(count)
                print(f"   {flag}: {count} ({percentage:.1f}%)")
    
    # Gene information
    if 'GENEINFO' in df.columns:
        unique_genes = df['GENEINFO'].nunique()
        print(f"\n🧬 GENE INFORMATION:")
        print(f"   Unique genes: {unique_genes}")
        if unique_genes <= 10:
            print(f"   Genes: {df['GENEINFO'].unique().tolist()}")
    
    if 'POS' in df.columns:
        print(f"\n📍 POSITION RANGE:")
        pos_min = int(df['POS'].min())
        pos_max = int(df['POS'].max())
        print(f"   Min position: {pos_min:,}")
        print(f"   Max position: {pos_max:,}")
        print(f"   Span: {pos_max - pos_min:,} bp")
        analysis['position_range'] = {
            'min': pos_min,
            'max': pos_max,
            'span': pos_max - pos_min
        }
    
    # Sample variants
    print(f"\n📝 SAMPLE VARIANTS (first 3):")
    sample_cols = ['CHROM', 'POS', 'REF', 'ALT', 'CLNSIG']
    available_cols = [col for col in sample_cols if col in df.columns]
    if available_cols:
        print(df[available_cols].head(3).to_string(index=False))
    
    return analysis


def analyze_gnomad(gnomad_json_path: str | Path) -> Dict:
    """
    Comprehensive analysis of gnomAD data
    
    Returns:
        Dict containing analysis results and the DataFrame
    """
    print("\n" + "="*80)
    print("GNOMAD ANALYSIS")
    print("="*80)
    
    df = load_json_data(gnomad_json_path)
    
    analysis = {
        'dataframe': df,
        'total_variants': len(df),
        'columns': list(df.columns),
        'shape': df.shape
    }
    
    # Basic statistics
    print(f"\n📊 BASIC STATISTICS:")
    print(f"   Total variant-transcript pairs: {len(df)}")
    print(f"   Columns: {df.shape[1]}")
    print(f"   Chromosomes: {df['CHROM'].unique().tolist() if 'CHROM' in df.columns else 'N/A'}")
    
    # Unique variants (before transcript expansion)
    if all(col in df.columns for col in ['CHROM', 'POS', 'REF', 'ALT']):
        unique_variants = df[['CHROM', 'POS', 'REF', 'ALT']].drop_duplicates()
        print(f"   Unique variants: {len(unique_variants)}")
        analysis['unique_variants'] = len(unique_variants)
    
    # Column summary
    print(f"\n📋 AVAILABLE COLUMNS ({len(df.columns)}):")
    print("   Core variant columns:")
    core_cols = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER']
    for col in core_cols:
        if col in df.columns:
            print(f"      - {col}")
    
    print("   Frequency columns:")
    freq_cols = [col for col in df.columns if 'AF' in col or 'AC' in col or 'AN' in col]
    for col in freq_cols[:10]:  # Show first 10
        print(f"      - {col}")
    if len(freq_cols) > 10:
        print(f"      ... and {len(freq_cols) - 10} more frequency columns")
    
    print("   VEP annotation columns:")
    vep_cols = [col for col in df.columns if col not in core_cols + freq_cols]
    for col in vep_cols[:15]:  # Show first 15
        print(f"      - {col}")
    if len(vep_cols) > 15:
        print(f"      ... and {len(vep_cols) - 15} more VEP columns")
    
    # Allele frequency analysis
    if 'AF' in df.columns:
        print(f"\n📈 ALLELE FREQUENCY STATISTICS:")
        af_series = pd.to_numeric(df['AF'], errors='coerce')
        analysis['allele_frequency'] = {
            'mean': float(af_series.mean()),
            'median': float(af_series.median()),
            'min': float(af_series.min()),
            'max': float(af_series.max())
        }
        print(f"   Mean AF: {af_series.mean():.6f}")
        print(f"   Median AF: {af_series.median():.6f}")
        print(f"   Min AF: {af_series.min():.6f}")
        print(f"   Max AF: {af_series.max():.6f}")
        
        # Frequency bins
        print(f"\n   Frequency distribution:")
        ultra_rare = (af_series < 0.00001).sum()
        rare = ((af_series >= 0.00001) & (af_series < 0.001)).sum()
        low_freq = ((af_series >= 0.001) & (af_series < 0.01)).sum()
        common = (af_series >= 0.01).sum()
        print(f"      Ultra-rare (AF < 0.00001): {ultra_rare} ({ultra_rare/len(df)*100:.1f}%)")
        print(f"      Rare (0.00001 ≤ AF < 0.001): {rare} ({rare/len(df)*100:.1f}%)")
        print(f"      Low frequency (0.001 ≤ AF < 0.01): {low_freq} ({low_freq/len(df)*100:.1f}%)")
        print(f"      Common (AF ≥ 0.01): {common} ({common/len(df)*100:.1f}%)")
    
    # Consequence analysis (VEP)
    if 'Consequence' in df.columns:
        print(f"\n🧬 VARIANT CONSEQUENCES:")
        consequence_counts = df['Consequence'].value_counts().head(10)
        analysis['consequences'] = consequence_counts.to_dict()
        for consequence, count in consequence_counts.items():
            percentage = (count / len(df)) * 100
            print(f"   {consequence}: {count} ({percentage:.1f}%)")
    
    # Impact analysis
    if 'IMPACT' in df.columns:
        print(f"\n⚡ VARIANT IMPACT:")
        impact_counts = df['IMPACT'].value_counts()
        analysis['impact'] = impact_counts.to_dict()
        for impact, count in impact_counts.items():
            percentage = (count / len(df)) * 100
            print(f"   {impact}: {count} ({percentage:.1f}%)")
    
    # Gene/transcript information
    if 'SYMBOL' in df.columns:
        unique_genes = df['SYMBOL'].nunique()
        print(f"\n🧬 GENE/TRANSCRIPT INFORMATION:")
        print(f"   Unique genes: {unique_genes}")
        if unique_genes <= 5:
            print(f"   Genes: {df['SYMBOL'].unique().tolist()}")
    
    if 'Feature' in df.columns:
        print(f"   Unique transcripts: {df['Feature'].nunique()}")
    
    # HGNC ID
    if 'HGNC_ID' in df.columns:
        print(f"   HGNC IDs: {df['HGNC_ID'].unique().tolist()}")
    
    # Sample variants
    print(f"\n📝 SAMPLE VARIANTS (first 3):")
    sample_cols = ['CHROM', 'POS', 'REF', 'ALT', 'AF', 'Consequence', 'IMPACT', 'HGVSp']
    available_cols = [col for col in sample_cols if col in df.columns]
    if available_cols:
        print(df[available_cols].head(3).to_string(index=False))
    
    return analysis


def compare_datasets(clinvar_df: pd.DataFrame, gnomad_df: pd.DataFrame) -> Dict:
    """
    Compare ClinVar and gnomAD datasets to find overlapping variants
    
    Returns:
        Dict with comparison statistics and merged data
    """
    print("\n" + "="*80)
    print("DATASET COMPARISON")
    print("="*80)
    
    # Ensure coordinate columns are strings for matching
    for df in [clinvar_df, gnomad_df]:
        for col in ['CHROM', 'POS', 'REF', 'ALT']:
            if col in df.columns:
                df[col] = df[col].astype(str)
    
    # Merge on variant coordinates
    merge_cols = ['CHROM', 'POS', 'REF', 'ALT']
    if all(col in clinvar_df.columns and col in gnomad_df.columns for col in merge_cols):
        
        # Get unique variants from gnomAD (it has transcript duplicates)
        gnomad_unique = gnomad_df[merge_cols + ['AF']].drop_duplicates(subset=merge_cols)
        
        merged = pd.merge(
            clinvar_df,
            gnomad_unique,
            on=merge_cols,
            how='outer',
            indicator=True,
            suffixes=('_clinvar', '_gnomad')
        )
        
        both = merged[merged['_merge'] == 'both']
        clinvar_only = merged[merged['_merge'] == 'left_only']
        gnomad_only = merged[merged['_merge'] == 'right_only']
        
        print(f"\n🔗 VARIANT OVERLAP:")
        print(f"   Total unique variants (union): {len(merged)}")
        print(f"   In both datasets: {len(both)} ({len(both)/len(merged)*100:.1f}%)")
        print(f"   ClinVar only: {len(clinvar_only)} ({len(clinvar_only)/len(merged)*100:.1f}%)")
        print(f"   gnomAD only: {len(gnomad_only)} ({len(gnomad_only)/len(merged)*100:.1f}%)")
        
        # Analyze overlapping variants
        if len(both) > 0:
            print(f"\n🎯 OVERLAPPING VARIANTS ANALYSIS:")
            
            # Clinical significance of overlapping variants
            if 'CLNSIG' in both.columns:
                print(f"\n   Clinical significance in overlapping variants:")
                for sig, count in both['CLNSIG'].value_counts().items():
                    print(f"      {sig}: {count}")
            
            # Allele frequency of pathogenic variants
            if 'is_pathogenic' in both.columns and 'AF' in both.columns:
                pathogenic = both[both['is_pathogenic'] == True]
                if len(pathogenic) > 0:
                    af_series = pd.to_numeric(pathogenic['AF'], errors='coerce')
                    print(f"\n   Pathogenic variants in gnomAD:")
                    print(f"      Count: {len(pathogenic)}")
                    print(f"      Mean AF: {af_series.mean():.6f}")
                    print(f"      Median AF: {af_series.median():.6f}")
                    
                    # Flag concerning variants (pathogenic but common)
                    concerning = pathogenic[af_series > 0.01]
                    if len(concerning) > 0:
                        print(f"\n   ⚠️  WARNING: {len(concerning)} pathogenic variants with AF > 1%")
                        print(f"      (These may be misclassified or benign)")
        
        comparison = {
            'merged_df': merged,
            'total_unique': len(merged),
            'in_both': len(both),
            'clinvar_only': len(clinvar_only),
            'gnomad_only': len(gnomad_only),
            'both_df': both
        }
        
        return comparison
    else:
        print("⚠️  Cannot compare: Missing coordinate columns in one or both datasets")
        return {'error': 'Missing coordinate columns'}


def generate_actionable_insights(clinvar_analysis: Dict, gnomad_analysis: Dict, 
                                 comparison: Optional[Dict] = None):
    """
    Generate actionable insights and recommendations based on the data
    """
    print("\n" + "="*80)
    print("ACTIONABLE INSIGHTS & RECOMMENDATIONS")
    print("="*80)
    
    clinvar_df = clinvar_analysis['dataframe']
    gnomad_df = gnomad_analysis['dataframe']
    
    print(f"\n💡 WHAT YOU CAN DO WITH THIS DATA:\n")
    
    # ClinVar insights
    print("1️⃣  CLINVAR DATA APPLICATIONS:")
    if 'is_pathogenic' in clinvar_df.columns:
        pathogenic_count = clinvar_df['is_pathogenic'].sum()
        print(f"   ✓ Identify {pathogenic_count} high-confidence pathogenic variants")
        print(f"   ✓ Use for genetic testing interpretation")
        print(f"   ✓ Prioritize variants for functional validation")
    
    if 'is_VUS' in clinvar_df.columns:
        vus_count = clinvar_df['is_VUS'].sum()
        print(f"   ✓ Found {vus_count} Variants of Uncertain Significance (VUS)")
        print(f"   ✓ Candidates for reclassification research")
    
    # gnomAD insights
    print(f"\n2️⃣  GNOMAD DATA APPLICATIONS:")
    if 'AF' in gnomad_df.columns:
        af_series = pd.to_numeric(gnomad_df['AF'], errors='coerce')
        rare_count = (af_series < 0.001).sum()
        print(f"   ✓ Filter {rare_count} rare variants (AF < 0.1%)")
        print(f"   ✓ Identify novel/ultra-rare variants for research")
        print(f"   ✓ Population-specific allele frequency analysis")
    
    if 'Consequence' in gnomad_df.columns:
        missense = (gnomad_df['Consequence'].str.contains('missense', case=False, na=False)).sum()
        print(f"   ✓ Analyze {missense} missense variants for pathogenicity prediction")
    
    # Combined insights
    if comparison and 'both_df' in comparison:
        both_df = comparison['both_df']
        print(f"\n3️⃣  INTEGRATED ANALYSIS:")
        print(f"   ✓ Cross-validate {len(both_df)} variants with both clinical and population data")
        
        if 'is_pathogenic' in both_df.columns and 'AF' in both_df.columns:
            pathogenic = both_df[both_df['is_pathogenic'] == True]
            af_series = pd.to_numeric(pathogenic['AF'], errors='coerce')
            rare_pathogenic = (af_series < 0.001).sum()
            print(f"   ✓ {rare_pathogenic} rare pathogenic variants (strong disease candidates)")
        
        print(f"   ✓ Identify potential ClinVar reclassification candidates")
        print(f"   ✓ Build pathogenicity prediction models")
    
    # Specific use cases
    print(f"\n4️⃣  RECOMMENDED ANALYSES:")
    print(f"   📊 Variant prioritization pipeline:")
    print(f"      1. Filter gnomAD for rare variants (AF < 0.001)")
    print(f"      2. Annotate with ClinVar pathogenicity")
    print(f"      3. Prioritize HIGH/MODERATE impact variants")
    print(f"      4. Focus on canonical transcripts")
    
    print(f"\n   🔬 Research applications:")
    print(f"      • Gene burden analysis")
    print(f"      • Rare variant association studies")
    print(f"      • Pathogenicity prediction training")
    print(f"      • Clinical variant reclassification")
    
    print(f"\n   🧬 Clinical applications:")
    print(f"      • Patient variant interpretation")
    print(f"      • Genetic testing result validation")
    print(f"      • Carrier screening design")
    print(f"      • Pharmacogenomic analysis")


def save_analysis_report(output_path: str | Path, clinvar_analysis: Dict, 
                        gnomad_analysis: Dict, comparison: Optional[Dict] = None):
    """Save analysis results to a text file"""
    output_path = Path(output_path)
    
    with open(output_path, 'w') as f:
        f.write("VARIANT DATA ANALYSIS REPORT\n")
        f.write("=" * 80 + "\n\n")
        
        f.write("CLINVAR SUMMARY\n")
        f.write("-" * 80 + "\n")
        f.write(f"Total variants: {clinvar_analysis['total_variants']}\n")
        f.write(f"Columns: {clinvar_analysis['shape'][1]}\n")
        if 'clinical_significance' in clinvar_analysis:
            f.write("\nClinical Significance:\n")
            for sig, count in clinvar_analysis['clinical_significance'].items():
                f.write(f"  {sig}: {count}\n")
        
        f.write("\n\nGNOMAD SUMMARY\n")
        f.write("-" * 80 + "\n")
        f.write(f"Total variant-transcript pairs: {gnomad_analysis['total_variants']}\n")
        f.write(f"Columns: {gnomad_analysis['shape'][1]}\n")
        if 'unique_variants' in gnomad_analysis:
            f.write(f"Unique variants: {gnomad_analysis['unique_variants']}\n")
        
        if comparison and 'total_unique' in comparison:
            f.write("\n\nCOMPARISON SUMMARY\n")
            f.write("-" * 80 + "\n")
            f.write(f"Total unique variants: {comparison['total_unique']}\n")
            f.write(f"In both datasets: {comparison['in_both']}\n")
            f.write(f"ClinVar only: {comparison['clinvar_only']}\n")
            f.write(f"gnomAD only: {comparison['gnomad_only']}\n")
    
    print(f"\n✅ Analysis report saved to: {output_path}")


def main():
    # ===== SET YOUR FILE PATHS HERE =====
    CLINVAR_JSON = ".cache/HBB_clinvar.json"  # Set to None if you don't want to analyze ClinVar
    GNOMAD_JSON = ".cache/gnomAD_test_HBB.json"      # Set to None if you don't want to analyze gnomAD
    OUTPUT_REPORT = None                          # Set to a path like "report.txt" to save report, or None to skip
    
    if not CLINVAR_JSON and not GNOMAD_JSON:
        raise ValueError("At least one of CLINVAR_JSON or GNOMAD_JSON must be specified")
    
    clinvar_analysis = None
    gnomad_analysis = None
    comparison = None
    
    try:
        # Analyze ClinVar
        if CLINVAR_JSON:
            clinvar_analysis = analyze_clinvar(CLINVAR_JSON)
        
        # Analyze gnomAD
        if GNOMAD_JSON:
            gnomad_analysis = analyze_gnomad(GNOMAD_JSON)
        
        # Compare if both provided
        if clinvar_analysis and gnomad_analysis:
            comparison = compare_datasets(
                clinvar_analysis['dataframe'],
                gnomad_analysis['dataframe']
            )
        
        # Generate insights
        if clinvar_analysis and gnomad_analysis:
            generate_actionable_insights(clinvar_analysis, gnomad_analysis, comparison)
        
        # Save report if requested
        if OUTPUT_REPORT:
            save_analysis_report(OUTPUT_REPORT, clinvar_analysis or {}, 
                               gnomad_analysis or {}, comparison)
        
        print("\n" + "="*80)
        print("✅ ANALYSIS COMPLETE")
        print("="*80 + "\n")
        
    except Exception as e:
        print(f"\n❌ ERROR: {e}", file=sys.stderr)
        import traceback
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    main()


CLINVAR ANALYSIS

📊 BASIC STATISTICS:
   Total variants: 593
   Columns: 37
   Chromosomes: ['11']

📋 AVAILABLE COLUMNS (37):
   - CHROM
   - POS
   - ID
   - REF
   - ALT
   - QUAL
   - FILTER
   - AC
   - AF
   - AF_ESP
   - AF_EXAC
   - AF_TGP
   - ALLELEID
   - AN
   - CLNDISDB
   - CLNDISDBINCL
   - CLNDN
   - CLNDNINCL
   - CLNHGVS
   - CLNREVSTAT
   - CLNSIG
   - CLNSIGCONF
   - CLNSIGINCL
   - CLNVC
   - CLNVCSO
   - CLNVI
   - DBVARID
   - DP
   - GENEINFO
   - MC
   - ORIGIN
   - RS
   - SSR
   - is_pathogenic
   - is_benign
   - is_conflicting
   - is_VUS

🔬 CLINICAL SIGNIFICANCE BREAKDOWN:
   other: 272 (45.9%)
   Pathogenic: 116 (19.6%)
   Uncertain_significance: 75 (12.6%)
   Pathogenic,_other: 42 (7.1%)
   Likely_benign: 25 (4.2%)
   Likely_pathogenic: 18 (3.0%)
   Benign: 12 (2.0%)
   Pathogenic/Likely_pathogenic: 10 (1.7%)
   Conflicting_interpretations_of_pathogenicity: 9 (1.5%)
   Conflicting_interpretations_of_pathogenicity,_other: 5 (0.8%)
   Benign/Likely_benign:

In [6]:
import pandas as pd
import json

# Load both datasets
with open('.cache/HBB_clinvar.json', 'r') as f:
    clinvar_data = json.load(f)
clinvar_df = pd.DataFrame(clinvar_data)

with open('.cache/gnomAD_test_HBB.json', 'r') as f:
    gnomad_data = json.load(f)
gnomad_df = pd.DataFrame(gnomad_data)


print("="*80)
print("MISSENSE VARIANT OVERLAP ANALYSIS")
print("="*80)

# Filter ClinVar for missense variants
clinvar_missense = clinvar_df[
    clinvar_df['CLNVC'].str.contains('missense', case=False, na=False) |
    clinvar_df['MC'].str.contains('missense', case=False, na=False)
].copy()

print(f"\n📊 ClinVar missense variants: {len(clinvar_missense)}")

# Filter gnomAD for missense variants
gnomad_missense = gnomad_df[
    gnomad_df['Consequence'].str.contains('missense', case=False, na=False)
].copy()

# Get unique gnomAD missense (remove transcript duplicates)
gnomad_missense_unique = gnomad_missense.drop_duplicates(
    subset=['CHROM', 'POS', 'REF', 'ALT']
)

print(f"📊 gnomAD missense variants: {len(gnomad_missense_unique)}")

# Standardize coordinates for merging
for df in [clinvar_missense, gnomad_missense_unique]:
    df['CHROM'] = df['CHROM'].astype(str)
    df['POS'] = df['POS'].astype(str)
    df['REF'] = df['REF'].astype(str)
    df['ALT'] = df['ALT'].astype(str)

# Merge to find overlap
merged_missense = pd.merge(
    clinvar_missense,
    gnomad_missense_unique,
    on=['CHROM', 'POS', 'REF', 'ALT'],
    how='outer',
    indicator=True,
    suffixes=('_clinvar', '_gnomad')
)

both = merged_missense[merged_missense['_merge'] == 'both']
clinvar_only = merged_missense[merged_missense['_merge'] == 'left_only']
gnomad_only = merged_missense[merged_missense['_merge'] == 'right_only']

print(f"\n🔗 MISSENSE VARIANT OVERLAP:")
print(f"   Total unique missense variants: {len(merged_missense)}")
print(f"   In both datasets: {len(both)} ({len(both)/len(merged_missense)*100:.1f}%)")
print(f"   ClinVar only: {len(clinvar_only)} ({len(clinvar_only)/len(merged_missense)*100:.1f}%)")
print(f"   gnomAD only: {len(gnomad_only)} ({len(gnomad_only)/len(merged_missense)*100:.1f}%)")

# Analyze overlapping missense variants
if len(both) > 0:
    print(f"\n🎯 OVERLAPPING MISSENSE VARIANTS:")
    
    # Clinical significance distribution
    if 'CLNSIG' in both.columns:
        print(f"\n   Clinical significance:")
        clnsig_counts = both['CLNSIG'].value_counts().head(10)
        for sig, count in clnsig_counts.items():
            print(f"      {sig}: {count}")
    
    # Check which AF column exists
    af_col = 'AF_gnomad' if 'AF_gnomad' in both.columns else 'AF' if 'AF' in both.columns else None
    
    # Allele frequencies of pathogenic missense
    if 'is_pathogenic' in both.columns and af_col:
        pathogenic_missense = both[both['is_pathogenic'] == True]
        if len(pathogenic_missense) > 0:
            af_series = pd.to_numeric(pathogenic_missense[af_col], errors='coerce').dropna()
            if len(af_series) > 0:
                print(f"\n   Pathogenic missense variants:")
                print(f"      Count: {len(pathogenic_missense)}")
                print(f"      Mean AF: {af_series.mean():.6f}")
                print(f"      Median AF: {af_series.median():.6f}")
                print(f"      Range: {af_series.min():.6f} - {af_series.max():.6f}")
            
            # Show examples with HGVSp
            print(f"\n   Example pathogenic missense variants:")
            example_cols = ['POS', 'REF', 'ALT', 'CLNSIG']
            if af_col:
                example_cols.append(af_col)
            if 'HGVSp_gnomad' in pathogenic_missense.columns:
                example_cols.append('HGVSp_gnomad')
            elif 'HGVSp' in pathogenic_missense.columns:
                example_cols.append('HGVSp')
            
            available_cols = [c for c in example_cols if c in pathogenic_missense.columns]
            if available_cols:
                print(pathogenic_missense[available_cols].head(10).to_string(index=False))

# Show ClinVar-only missense
print(f"\n📋 CLINVAR-ONLY MISSENSE (no population data):")
print(f"   Count: {len(clinvar_only)}")
if len(clinvar_only) > 0 and 'CLNSIG' in clinvar_only.columns:
    print(f"\n   Clinical significance breakdown:")
    clnsig_only = clinvar_only['CLNSIG'].value_counts().head(5)
    for sig, count in clnsig_only.items():
        print(f"      {sig}: {count}")
    print(f"\n   Interpretation:")
    print(f"      • Ultra-rare private mutations")
    print(f"      • Not seen in gnomAD populations")
    print(f"      • Potentially novel disease variants")

# Show gnomAD-only missense
print(f"\n📋 GNOMAD-ONLY MISSENSE (no clinical annotation):")
print(f"   Count: {len(gnomad_only)}")

if len(gnomad_only) > 0:
    # Check which AF column exists in gnomad_only
    af_col_gnomad = None
    for col_name in ['AF_gnomad', 'AF']:
        if col_name in gnomad_only.columns:
            af_col_gnomad = col_name
            break
    
    if af_col_gnomad:
        af_series = pd.to_numeric(gnomad_only[af_col_gnomad], errors='coerce').dropna()
        if len(af_series) > 0:
            print(f"   Mean AF: {af_series.mean():.6f}")
            print(f"   Median AF: {af_series.median():.6f}")
            rare_count = (af_series < 0.001).sum()
            common_count = (af_series >= 0.01).sum()
            print(f"   Rare variants (AF < 0.1%): {rare_count}")
            print(f"   Common variants (AF ≥ 1%): {common_count}")
    
    print(f"\n   Interpretation:")
    print(f"      • Likely benign population variants")
    print(f"      • VUS candidates needing clinical interpretation")
    print(f"      • Common polymorphisms")

print("\n" + "="*80)

# Summary statistics
print("\nSUMMARY:")
print(f"✓ ClinVar missense: {len(clinvar_missense)}")
print(f"✓ gnomAD missense: {len(gnomad_missense_unique)}")
print(f"✓ Overlap: {len(both)} ({len(both)/len(merged_missense)*100:.1f}%)")
print(f"✓ ClinVar pathogenic in overlap: {both['is_pathogenic'].sum() if 'is_pathogenic' in both.columns else 'N/A'}")
print(f"✓ Gold standard training pairs: {len(both)}")
print("="*80)

MISSENSE VARIANT OVERLAP ANALYSIS

📊 ClinVar missense variants: 383
📊 gnomAD missense variants: 203

🔗 MISSENSE VARIANT OVERLAP:
   Total unique missense variants: 481
   In both datasets: 105 (21.8%)
   ClinVar only: 278 (57.8%)
   gnomAD only: 98 (20.4%)

🎯 OVERLAPPING MISSENSE VARIANTS:

   Clinical significance:
      other: 50
      Uncertain_significance: 17
      Pathogenic: 12
      Likely_benign: 9
      Conflicting_interpretations_of_pathogenicity,_other: 5
      Pathogenic,_other: 4
      Benign: 3
      Likely_pathogenic: 3
      Conflicting_interpretations_of_pathogenicity: 2

   Pathogenic missense variants:
      Count: 11
      Mean AF: 0.001543
      Median AF: 0.000020
      Range: 0.000001 - 0.012719

   Example pathogenic missense variants:
    POS REF ALT            CLNSIG  AF_gnomad                         HGVSp
5225662   A   C        Pathogenic   0.000002 ENSP00000333994.3:p.Val127Gly
5225678   C   T        Pathogenic   0.000033 ENSP00000333994.3:p.Glu122Lys
5225

/var/tmp/ipykernel_99819/4001673238.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CHROM'] = df['CHROM'].astype(str)
/var/tmp/ipykernel_99819/4001673238.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['POS'] = df['POS'].astype(str)
/var/tmp/ipykernel_99819/4001673238.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [9]:
import pandas as pd
import json
import re
import requests
import time
from typing import Dict, Optional
from pathlib import Path


def parse_hgvsp_notation(hgvsp: str) -> Optional[Dict]:
    """
    Parse HGVSp notation to extract ref_aa, position, alt_aa
    
    Handles formats:
        ENSP00000333994.3:p.Glu6Val → {'ref_aa': 'E', 'pos': 6, 'alt_aa': 'V'}
        p.Glu6Val → {'ref_aa': 'E', 'pos': 6, 'alt_aa': 'V'}
        NP_000509.1:p.His144Arg → {'ref_aa': 'H', 'pos': 144, 'alt_aa': 'R'}
    """
    if pd.isna(hgvsp) or hgvsp == '':
        return None
    
    # Three-letter to one-letter amino acid code
    aa_3to1 = {
        'Ala': 'A', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cys': 'C',
        'Gln': 'Q', 'Glu': 'E', 'Gly': 'G', 'His': 'H', 'Ile': 'I',
        'Leu': 'L', 'Lys': 'K', 'Met': 'M', 'Phe': 'F', 'Pro': 'P',
        'Ser': 'S', 'Thr': 'T', 'Trp': 'W', 'Tyr': 'Y', 'Val': 'V',
        'Ter': '*', 'Stop': '*', 'Sec': 'U', 'Pyl': 'O'
    }
    
    # Strip protein ID prefix if present (ENSP...:p. or NP_...:p.)
    hgvsp_str = str(hgvsp)
    if ':p.' in hgvsp_str:
        hgvsp_str = 'p.' + hgvsp_str.split(':p.')[1]
    
    # Match: p.Glu6Val or p.E6V or p.Glu6*
    pattern = r'p\.([A-Z][a-z]{2}|[A-Z\*])(\d+)([A-Z][a-z]{2}|[A-Z\*\=])'
    match = re.match(pattern, hgvsp_str)
    
    if match:
        ref_aa = match.group(1)
        pos = int(match.group(2))
        alt_aa = match.group(3)
        
        # Convert to single letter
        ref_1letter = aa_3to1.get(ref_aa, ref_aa)
        alt_1letter = aa_3to1.get(alt_aa, alt_aa) if alt_aa != '=' else ref_aa
        
        return {
            'ref_aa': ref_1letter,
            'pos': pos,
            'alt_aa': alt_1letter
        }
    
    return None


def get_protein_sequence_from_ensembl(transcript_id: str) -> Optional[str]:
    """
    Fetch protein sequence from Ensembl REST API
    """
    server = "https://rest.ensembl.org"
    ext = f"/sequence/id/{transcript_id}?type=protein"
    
    try:
        response = requests.get(
            server + ext, 
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        response.raise_for_status()
        data = response.json()
        return data['seq']
    except Exception as e:
        print(f"  ⚠️  Error fetching {transcript_id}: {e}")
        return None


def process_gnomad_json(json_path: str, 
                        fetch_sequences: bool = True,
                        canonical_only: bool = True) -> pd.DataFrame:
    """
    Process gnomAD JSON file to extract protein-level information
    
    Args:
        json_path: Path to gnomAD JSON file
        fetch_sequences: Whether to fetch protein sequences from Ensembl
        canonical_only: Keep only canonical transcripts
    
    Returns:
        DataFrame with protein-level variant information
    """
    
    print("="*80)
    print("PROCESSING GNOMAD JSON")
    print("="*80)
    
    # Load JSON
    print(f"\n📂 Loading: {json_path}")
    with open(json_path, 'r') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    print(f"   Total rows: {len(df)}")
    
    # Filter for missense variants only
    print(f"\n🔍 Filtering for missense variants...")
    missense = df[
        df['Consequence'].str.contains('missense', case=False, na=False)
    ].copy()
    print(f"   Missense variants: {len(missense)}")
    
    if len(missense) == 0:
        print("❌ No missense variants found!")
        return pd.DataFrame()
    
    # Parse HGVSp notation
    print(f"\n🧬 Parsing HGVSp notation...")
    missense['protein_change'] = missense['HGVSp'].apply(parse_hgvsp_notation)
    
    # Extract individual fields
    missense['ref_aa'] = missense['protein_change'].apply(
        lambda x: x['ref_aa'] if x else None
    )
    missense['protein_pos'] = missense['protein_change'].apply(
        lambda x: x['pos'] if x else None
    )
    missense['alt_aa'] = missense['protein_change'].apply(
        lambda x: x['alt_aa'] if x else None
    )
    
    # Create mutation string
    missense['mutation'] = missense.apply(
        lambda row: f"{row['ref_aa']}{row['protein_pos']}{row['alt_aa']}" 
        if row['ref_aa'] else None, 
        axis=1
    )
    
    # Remove failed parses
    parsed = missense[missense['ref_aa'].notna()].copy()
    print(f"   Successfully parsed: {len(parsed)}")
    print(f"   Failed to parse: {len(missense) - len(parsed)}")
    
    # Filter for canonical transcripts if requested
    if canonical_only:
        print(f"\n📌 Filtering for canonical transcripts...")
        before = len(parsed)
        parsed = parsed[parsed['CANONICAL'] == 'YES'].copy()
        print(f"   Before: {before}")
        print(f"   After: {len(parsed)}")
    
    # Select key columns
    key_columns = [
        'CHROM', 'POS', 'REF', 'ALT',
        'SYMBOL', 'Gene', 'Feature', 'Feature_type',
        'CANONICAL', 'BIOTYPE',
        'ref_aa', 'protein_pos', 'alt_aa', 'mutation',
        'HGVSp', 'HGVSc', 'Amino_acids', 'Codons',
        'AF', 'AC', 'AN', 'IMPACT', 'Consequence'
    ]
    
    # Add optional columns if they exist
    optional_columns = ['MANE_SELECT', 'SIFT_pred', 'PolyPhen_pred', 'EXON', 'INTRON']
    for col in optional_columns:
        if col in parsed.columns:
            key_columns.append(col)
    
    # Select available columns
    available_columns = [col for col in key_columns if col in parsed.columns]
    result = parsed[available_columns].copy()
    
    # Fetch protein sequences if requested
    if fetch_sequences:
        print(f"\n🔬 Fetching protein sequences from Ensembl...")
        
        # Get unique transcripts
        unique_transcripts = result['Feature'].dropna().unique()
        print(f"   Unique transcripts: {len(unique_transcripts)}")
        
        # Fetch sequences
        transcript_to_seq = {}
        for i, transcript_id in enumerate(unique_transcripts, 1):
            print(f"   [{i}/{len(unique_transcripts)}] Fetching {transcript_id}...")
            seq = get_protein_sequence_from_ensembl(transcript_id)
            transcript_to_seq[transcript_id] = seq
            time.sleep(0.1)  # Rate limiting
        
        # Map to dataframe
        result['protein_seq'] = result['Feature'].map(transcript_to_seq)
        
        # Verify sequences
        print(f"\n✅ Verifying protein sequences...")
        def verify_match(row):
            if pd.isna(row['protein_seq']) or pd.isna(row['protein_pos']):
                return None
            try:
                actual_aa = row['protein_seq'][row['protein_pos'] - 1]
                expected_aa = row['ref_aa']
                return actual_aa == expected_aa
            except:
                return None
        
        result['seq_verified'] = result.apply(verify_match, axis=1)
        
        verified = result['seq_verified'].sum()
        failed = (result['seq_verified'] == False).sum()
        missing = result['seq_verified'].isna().sum()
        
        print(f"   Verified matches: {verified}")
        print(f"   Mismatches: {failed}")
        print(f"   Unable to verify: {missing}")
        
        if failed > 0:
            print(f"\n   ⚠️  {failed} mismatches found (likely due to isoforms/versions)")
    
    # Remove duplicates at genomic level
    print(f"\n🔄 Removing duplicate genomic positions...")
    before_dedup = len(result)
    result_unique = result.drop_duplicates(subset=['CHROM', 'POS', 'REF', 'ALT']).copy()
    print(f"   Before: {before_dedup}")
    print(f"   After: {len(result_unique)}")
    
    # Summary
    print(f"\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print(f"✓ Total missense variants: {len(result_unique)}")
    print(f"✓ Genes: {result_unique['SYMBOL'].nunique()}")
    print(f"✓ Transcripts: {result_unique['Feature'].nunique()}")
    if 'protein_seq' in result_unique.columns:
        print(f"✓ With protein sequences: {result_unique['protein_seq'].notna().sum()}")
    print("="*80 + "\n")
    
    return result_unique


def main():
    """
    Main function - Process gnomAD JSON file
    """
    
    # ===== CONFIGURATION =====
    INPUT_JSON = ".cache/gnomAD_test_HBB.json"  # Change this to your file
    OUTPUT_CSV = ".cache/HBB_protein_level.csv"
    OUTPUT_JSON = ".cache/HBB_protein_level.json"
    
    FETCH_SEQUENCES = True      # Set to False to skip Ensembl API calls
    CANONICAL_ONLY = True       # Set to False to keep all transcripts
    
    # ===== PROCESS =====
    result = process_gnomad_json(
        INPUT_JSON,
        fetch_sequences=FETCH_SEQUENCES,
        canonical_only=CANONICAL_ONLY
    )
    
    if len(result) == 0:
        print("❌ No data to save!")
        return
    
    # ===== SAVE =====
    print(f"💾 Saving results...")
    result.to_csv(OUTPUT_CSV, index=False)
    print(f"   ✓ CSV: {OUTPUT_CSV}")
    
    result.to_json(OUTPUT_JSON, orient='records', indent=2)
    print(f"   ✓ JSON: {OUTPUT_JSON}")
    
    # ===== DISPLAY SAMPLE =====
    print(f"\n📊 Sample output:")
    display_cols = ['CHROM', 'POS', 'mutation', 'HGVSp', 'AF', 'Feature']
    available_display = [col for col in display_cols if col in result.columns]
    print(result[available_display].head(10).to_string(index=False))
    
    print(f"\n✅ DONE!")


if __name__ == "__main__":
    main()

PROCESSING GNOMAD JSON

📂 Loading: .cache/gnomAD_test_HBB.json
   Total rows: 8946

🔍 Filtering for missense variants...
   Missense variants: 1057

🧬 Parsing HGVSp notation...
   Successfully parsed: 1057
   Failed to parse: 0

📌 Filtering for canonical transcripts...
   Before: 1057
   After: 432

🔬 Fetching protein sequences from Ensembl...
   Unique transcripts: 2
   [1/2] Fetching ENST00000335295...
   [2/2] Fetching NM_000518.5...
  ⚠️  Error fetching NM_000518.5: 400 Client Error: Bad Request for url: https://rest.ensembl.org/sequence/id/NM_000518.5?type=protein

✅ Verifying protein sequences...
   Verified matches: 216
   Mismatches: 0
   Unable to verify: 216

🔄 Removing duplicate genomic positions...
   Before: 432
   After: 170

SUMMARY
✓ Total missense variants: 170
✓ Genes: 1
✓ Transcripts: 1
✓ With protein sequences: 170

💾 Saving results...
   ✓ CSV: .cache/HBB_protein_level.csv
   ✓ JSON: .cache/HBB_protein_level.json

📊 Sample output:
CHROM     POS mutation            

In [8]:
import pandas as pd
import json

# Quick diagnostic
with open('.cache/gnomAD_test_HBB.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Check missense variants
missense = df[df['Consequence'].str.contains('missense', case=False, na=False)]

print(f"Total missense: {len(missense)}")
print(f"\nColumns available:")
print(missense.columns.tolist())

print(f"\nSample HGVSp values:")
print(missense['HGVSp'].head(20).tolist())

print(f"\nHGVSp data types:")
print(missense['HGVSp'].apply(type).value_counts())

print(f"\nSample rows:")
print(missense[['Consequence', 'HGVSp', 'Amino_acids', 'Feature']].head(10))

Total missense: 1057

Columns available:
['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'AC', 'AF', 'vep', 'index', 'Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'ALLELE_NUM', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'UNIPROT_ISOFORM', 'SOURCE', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'TRANSCRIPTION_FACTORS', 'LoF', 'LoF_filter', 'LoF_flags', 'LoF_info']

Sample HGVSp values:
['ENSP00000333994.3:p.His144Arg', 'ENSP00000494175.1:p.His144Arg', 'NP_000509.1:p.His144Arg', 'ENSP00000333994.3:p.His144Tyr', 'ENSP00000494175.1:p.His144Tyr', 'NP_000509.1:p.His144Tyr', 'ENSP00000333994.3:p.His144Tyr', 'ENSP00000494175.1:p.His144Tyr', 'NP_000509.1:

In [ ]:
import pandas as pd
import re
from collections import Counter

def validate_protein_level_data(csv_path: str):
    """
    Comprehensive validation of protein-level missense data
    """
    
    print("="*80)
    print("COMPREHENSIVE DATA VALIDATION")
    print("="*80)
    
    # Load data
    print(f"\n📂 Loading: {csv_path}")
    df = pd.read_csv(csv_path)
    print(f"   Total rows: {len(df)}")
    
    errors = []
    warnings = []
    
    # ============================================
    # 1. CHECK REQUIRED COLUMNS
    # ============================================
    print("\n1️⃣  CHECKING REQUIRED COLUMNS...")
    required_cols = [
        'CHROM', 'POS', 'REF', 'ALT',
        'ref_aa', 'protein_pos', 'alt_aa', 'mutation',
        'Feature', 'protein_seq'
    ]
    
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        errors.append(f"Missing columns: {missing_cols}")
        print(f"   ❌ Missing columns: {missing_cols}")
    else:
        print(f"   ✅ All required columns present")
    
    # ============================================
    # 2. CHECK FOR NULL VALUES
    # ============================================
    print("\n2️⃣  CHECKING FOR NULL VALUES...")
    critical_cols = ['ref_aa', 'protein_pos', 'alt_aa', 'protein_seq']
    
    for col in critical_cols:
        if col in df.columns:
            null_count = df[col].isna().sum()
            if null_count > 0:
                errors.append(f"{col} has {null_count} null values")
                print(f"   ❌ {col}: {null_count} nulls")
            else:
                print(f"   ✅ {col}: no nulls")
    
    # ============================================
    # 3. VALIDATE AMINO ACIDS
    # ============================================
    print("\n3️⃣  VALIDATING AMINO ACIDS...")
    valid_aas = set('ACDEFGHIKLMNPQRSTVWY*')
    
    invalid_ref = df[~df['ref_aa'].isin(valid_aas)]
    invalid_alt = df[~df['alt_aa'].isin(valid_aas)]
    
    if len(invalid_ref) > 0:
        errors.append(f"{len(invalid_ref)} invalid ref_aa")
        print(f"   ❌ Invalid ref_aa: {invalid_ref['ref_aa'].unique()}")
    else:
        print(f"   ✅ All ref_aa valid")
    
    if len(invalid_alt) > 0:
        errors.append(f"{len(invalid_alt)} invalid alt_aa")
        print(f"   ❌ Invalid alt_aa: {invalid_alt['alt_aa'].unique()}")
    else:
        print(f"   ✅ All alt_aa valid")
    
    # ============================================
    # 4. VERIFY PROTEIN SEQUENCES
    # ============================================
    print("\n4️⃣  VERIFYING PROTEIN SEQUENCES MATCH...")
    
    mismatches = []
    seq_errors = []
    verified = 0
    
    for idx, row in df.iterrows():
        if pd.notna(row['protein_seq']) and pd.notna(row['protein_pos']):
            try:
                # Check position is within bounds
                if row['protein_pos'] < 1 or row['protein_pos'] > len(row['protein_seq']):
                    seq_errors.append(f"Row {idx}: position {row['protein_pos']} out of bounds (seq length: {len(row['protein_seq'])})")
                    continue
                
                # Check ref_aa matches sequence
                actual_aa = row['protein_seq'][row['protein_pos'] - 1]
                expected_aa = row['ref_aa']
                
                if actual_aa == expected_aa:
                    verified += 1
                else:
                    mismatches.append({
                        'row': idx,
                        'position': row['protein_pos'],
                        'expected': expected_aa,
                        'actual': actual_aa,
                        'mutation': row['mutation']
                    })
            except Exception as e:
                seq_errors.append(f"Row {idx}: {e}")
    
    print(f"   ✅ Verified matches: {verified}/{len(df)}")
    
    if mismatches:
        errors.append(f"{len(mismatches)} sequence mismatches")
        print(f"   ❌ Sequence mismatches: {len(mismatches)}")
        print(f"\n   Sample mismatches:")
        for mm in mismatches[:5]:
            print(f"      Row {mm['row']}: {mm['mutation']} - Expected {mm['expected']} but got {mm['actual']} at pos {mm['position']}")
    
    if seq_errors:
        errors.append(f"{len(seq_errors)} sequence validation errors")
        print(f"   ❌ Validation errors: {len(seq_errors)}")
        for err in seq_errors[:5]:
            print(f"      {err}")
    
    # ============================================
    # 5. CHECK MUTATION STRING FORMAT
    # ============================================
    print("\n5️⃣  VALIDATING MUTATION STRINGS...")
    
    mutation_pattern = r'^[ACDEFGHIKLMNPQRSTVWY\*]\d+[ACDEFGHIKLMNPQRSTVWY\*]$'
    invalid_mutations = df[~df['mutation'].str.match(mutation_pattern, na=False)]
    
    if len(invalid_mutations) > 0:
        errors.append(f"{len(invalid_mutations)} invalid mutation formats")
        print(f"   ❌ Invalid formats: {len(invalid_mutations)}")
        print(f"      Examples: {invalid_mutations['mutation'].head().tolist()}")
    else:
        print(f"   ✅ All mutation strings valid")
    
    # Verify mutation string matches components
    mutation_mismatch = 0
    for idx, row in df.iterrows():
        expected = f"{row['ref_aa']}{row['protein_pos']}{row['alt_aa']}"
        if row['mutation'] != expected:
            mutation_mismatch += 1
    
    if mutation_mismatch > 0:
        errors.append(f"{mutation_mismatch} mutation strings don't match components")
        print(f"   ❌ Mutation string mismatches: {mutation_mismatch}")
    else:
        print(f"   ✅ All mutation strings match components")
    
    # ============================================
    # 6. CHECK GENOMIC COORDINATES
    # ============================================
    print("\n6️⃣  VALIDATING GENOMIC COORDINATES...")
    
    # Check chromosomes
    valid_chroms = set([str(i) for i in range(1, 23)] + ['X', 'Y', 'M', 'MT'])
    invalid_chroms = df[~df['CHROM'].astype(str).isin(valid_chroms)]
    
    if len(invalid_chroms) > 0:
        warnings.append(f"{len(invalid_chroms)} non-standard chromosomes")
        print(f"   ⚠️  Non-standard chroms: {invalid_chroms['CHROM'].unique()}")
    else:
        print(f"   ✅ All chromosomes valid")
    
    # Check positions are positive
    invalid_pos = df[df['POS'] <= 0]
    if len(invalid_pos) > 0:
        errors.append(f"{len(invalid_pos)} invalid positions")
        print(f"   ❌ Invalid positions: {len(invalid_pos)}")
    else:
        print(f"   ✅ All positions > 0")
    
    # Check REF/ALT are valid nucleotides
    valid_nucs = set('ACGT')
    invalid_ref = df[~df['REF'].str.match(r'^[ACGT]+$', na=False)]
    invalid_alt = df[~df['ALT'].str.match(r'^[ACGT]+$', na=False)]
    
    if len(invalid_ref) > 0:
        errors.append(f"{len(invalid_ref)} invalid REF alleles")
        print(f"   ❌ Invalid REF: {len(invalid_ref)}")
    
    if len(invalid_alt) > 0:
        errors.append(f"{len(invalid_alt)} invalid ALT alleles")
        print(f"   ❌ Invalid ALT: {len(invalid_alt)}")
    
    # ============================================
    # 7. CHECK FOR DUPLICATES
    # ============================================
    print("\n7️⃣  CHECKING FOR DUPLICATES...")
    
    # Genomic duplicates
    genomic_dups = df[df.duplicated(subset=['CHROM', 'POS', 'REF', 'ALT'], keep=False)]
    if len(genomic_dups) > 0:
        errors.append(f"{len(genomic_dups)} duplicate genomic positions")
        print(f"   ❌ Genomic duplicates: {len(genomic_dups)}")
        print(f"      Example: {genomic_dups[['CHROM', 'POS', 'REF', 'ALT']].head()}")
    else:
        print(f"   ✅ No genomic duplicates")
    
    # Protein duplicates (same mutation)
    protein_dups = df[df.duplicated(subset=['Feature', 'mutation'], keep=False)]
    if len(protein_dups) > 0:
        warnings.append(f"{len(protein_dups)} duplicate protein mutations")
        print(f"   ⚠️  Protein duplicates: {len(protein_dups)}")
    else:
        print(f"   ✅ No protein duplicates")
    
    # ============================================
    # 8. VALIDATE ALLELE FREQUENCIES
    # ============================================
    print("\n8️⃣  VALIDATING ALLELE FREQUENCIES...")
    
    if 'AF' in df.columns:
        af_series = pd.to_numeric(df['AF'], errors='coerce')
        
        # Check range [0, 1]
        invalid_af = df[(af_series < 0) | (af_series > 1)]
        if len(invalid_af) > 0:
            errors.append(f"{len(invalid_af)} AF values out of range [0,1]")
            print(f"   ❌ Invalid AF range: {len(invalid_af)}")
        else:
            print(f"   ✅ All AF in range [0, 1]")
        
        # Show distribution
        print(f"\n   AF Distribution:")
        print(f"      Ultra-rare (< 0.00001): {(af_series < 0.00001).sum()}")
        print(f"      Rare (0.00001-0.001): {((af_series >= 0.00001) & (af_series < 0.001)).sum()}")
        print(f"      Low freq (0.001-0.01): {((af_series >= 0.001) & (af_series < 0.01)).sum()}")
        print(f"      Common (> 0.01): {(af_series >= 0.01).sum()}")
    
    # ============================================
    # 9. CHECK TRANSCRIPT CONSISTENCY
    # ============================================
    print("\n9️⃣  CHECKING TRANSCRIPT CONSISTENCY...")
    
    unique_transcripts = df['Feature'].nunique()
    print(f"   Unique transcripts: {unique_transcripts}")
    
    if unique_transcripts > 1:
        warnings.append(f"Multiple transcripts: {df['Feature'].unique().tolist()}")
        print(f"   ⚠️  Multiple transcripts (expected if not canonical_only)")
        for transcript in df['Feature'].unique():
            count = (df['Feature'] == transcript).sum()
            print(f"      {transcript}: {count} variants")
    else:
        print(f"   ✅ Single transcript (canonical)")
    
    # Check all variants from same transcript have same sequence
    for transcript in df['Feature'].unique():
        transcript_df = df[df['Feature'] == transcript]
        unique_seqs = transcript_df['protein_seq'].nunique()
        if unique_seqs > 1:
            errors.append(f"Transcript {transcript} has {unique_seqs} different sequences")
            print(f"   ❌ {transcript}: {unique_seqs} different sequences!")
    
    # ============================================
    # 10. STATISTICAL SUMMARY
    # ============================================
    print("\n🔟 STATISTICAL SUMMARY...")
    
    print(f"   Protein position range: {df['protein_pos'].min()} - {df['protein_pos'].max()}")
    print(f"   Protein length: {len(df['protein_seq'].iloc[0]) if len(df) > 0 else 'N/A'}")
    print(f"   Unique positions affected: {df['protein_pos'].nunique()}")
    print(f"   Unique mutations: {df['mutation'].nunique()}")
    
    # Most common amino acid changes
    print(f"\n   Most common substitutions:")
    ref_alt = df['ref_aa'] + '→' + df['alt_aa']
    for sub, count in ref_alt.value_counts().head(5).items():
        print(f"      {sub}: {count}")
    
    # ============================================
    # FINAL REPORT
    # ============================================
    print("\n" + "="*80)
    print("VALIDATION SUMMARY")
    print("="*80)
    
    if len(errors) == 0 and len(warnings) == 0:
        print("\n🎉 ✅ DATA IS 100% CORRECT!")
        print(f"\n   Total validated variants: {len(df)}")
        print(f"   All checks passed")
        return True
    else:
        if errors:
            print(f"\n❌ CRITICAL ERRORS ({len(errors)}):")
            for i, err in enumerate(errors, 1):
                print(f"   {i}. {err}")
        
        if warnings:
            print(f"\n⚠️  WARNINGS ({len(warnings)}):")
            for i, warn in enumerate(warnings, 1):
                print(f"   {i}. {warn}")
        
        print(f"\n   Action required: Review and fix errors before proceeding")
        return False


# RUN VALIDATION
if __name__ == "__main__":
    validate_protein_level_data('.cache/HBB_protein_level.csv')

COMPREHENSIVE DATA VALIDATION

📂 Loading: .cache/HBB_protein_level.csv
   Total rows: 170

1️⃣  CHECKING REQUIRED COLUMNS...
   ✅ All required columns present

2️⃣  CHECKING FOR NULL VALUES...
   ✅ ref_aa: no nulls
   ✅ protein_pos: no nulls
   ✅ alt_aa: no nulls
   ✅ protein_seq: no nulls

3️⃣  VALIDATING AMINO ACIDS...
   ✅ All ref_aa valid
   ✅ All alt_aa valid

4️⃣  VERIFYING PROTEIN SEQUENCES MATCH...
   ✅ Verified matches: 170/170

5️⃣  VALIDATING MUTATION STRINGS...
   ✅ All mutation strings valid
   ✅ All mutation strings match components

6️⃣  VALIDATING GENOMIC COORDINATES...
   ✅ All chromosomes valid
   ✅ All positions > 0

7️⃣  CHECKING FOR DUPLICATES...
   ✅ No genomic duplicates
   ⚠️  Protein duplicates: 8

8️⃣  VALIDATING ALLELE FREQUENCIES...
   ✅ All AF in range [0, 1]

   AF Distribution:
      Ultra-rare (< 0.00001): 139
      Rare (0.00001-0.001): 29
      Low freq (0.001-0.01): 1
      Common (> 0.01): 1

9️⃣  CHECKING TRANSCRIPT CONSISTENCY...
   Unique transcri

In [11]:
import pandas as pd
import requests
import time
from typing import Dict, Optional, List
from Bio import SeqIO
from io import StringIO

def fetch_uniprot_sequence(gene_symbol: str) -> Optional[Dict]:
    """
    Fetch canonical protein sequence from UniProt
    """
    print(f"\n🔬 Querying UniProt for {gene_symbol}...")
    
    # Search for gene
    search_url = f"https://rest.uniprot.org/uniprotkb/search?query=gene:{gene_symbol}+AND+organism_id:9606+AND+reviewed:true&format=json"
    
    try:
        response = requests.get(search_url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if 'results' not in data or len(data['results']) == 0:
            print(f"   ⚠️  No UniProt entry found")
            return None
        
        # Get first (canonical) entry
        entry = data['results'][0]
        accession = entry['primaryAccession']
        sequence = entry['sequence']['value']
        length = entry['sequence']['length']
        
        print(f"   ✅ Found: {accession}")
        print(f"   Length: {length} aa")
        
        return {
            'accession': accession,
            'sequence': sequence,
            'length': length,
            'gene': entry.get('genes', [{}])[0].get('geneName', {}).get('value', gene_symbol)
        }
    
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return None


def fetch_refseq_sequence(gene_symbol: str) -> Optional[Dict]:
    """
    Fetch protein sequence from NCBI RefSeq
    """
    print(f"\n🧬 Querying NCBI RefSeq for {gene_symbol}...")
    
    try:
        from Bio import Entrez
        Entrez.email = "your.email@example.com"  # REQUIRED by NCBI
        
        # Search for gene
        search_handle = Entrez.esearch(
            db="gene",
            term=f"{gene_symbol}[Gene] AND human[Organism]",
            retmax=1
        )
        search_results = Entrez.read(search_handle)
        search_handle.close()
        
        if not search_results['IdList']:
            print(f"   ⚠️  No gene found")
            return None
        
        gene_id = search_results['IdList'][0]
        
        # Get gene details
        summary_handle = Entrez.esummary(db="gene", id=gene_id)
        summary = Entrez.read(summary_handle)
        summary_handle.close()
        
        # Try to get protein accession from gene
        # This is simplified - real implementation would need more parsing
        print(f"   Gene ID: {gene_id}")
        print(f"   ⚠️  RefSeq protein lookup requires additional parsing")
        
        return None
        
    except ImportError:
        print(f"   ⚠️  Biopython not installed (pip install biopython)")
        return None
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return None


def fetch_ensembl_gene_info(gene_symbol: str) -> Optional[Dict]:
    """
    Fetch gene and transcript info from Ensembl
    """
    print(f"\n🧪 Querying Ensembl for {gene_symbol}...")
    
    try:
        # Look up gene
        lookup_url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{gene_symbol}"
        response = requests.get(
            lookup_url,
            headers={"Content-Type": "application/json"},
            timeout=10
        )
        response.raise_for_status()
        gene_data = response.json()
        
        gene_id = gene_data['id']
        print(f"   Gene ID: {gene_id}")
        
        # Get canonical transcript
        if 'canonical_transcript' in gene_data:
            canonical = gene_data['canonical_transcript']
            print(f"   Canonical transcript: {canonical}")
            
            # Fetch sequence
            seq_url = f"https://rest.ensembl.org/sequence/id/{canonical}?type=protein"
            seq_response = requests.get(
                seq_url,
                headers={"Content-Type": "application/json"},
                timeout=10
            )
            seq_response.raise_for_status()
            seq_data = seq_response.json()
            
            sequence = seq_data['seq']
            print(f"   Length: {len(sequence)} aa")
            
            return {
                'gene_id': gene_id,
                'transcript_id': canonical,
                'sequence': sequence,
                'length': len(sequence),
                'chromosome': gene_data.get('seq_region_name'),
                'start': gene_data.get('start'),
                'end': gene_data.get('end')
            }
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return None


def compare_sequences(seq1: str, seq2: str, name1: str, name2: str) -> Dict:
    """
    Compare two protein sequences
    """
    if seq1 == seq2:
        return {
            'identical': True,
            'identity': 100.0,
            'differences': []
        }
    
    # Find differences
    differences = []
    min_len = min(len(seq1), len(seq2))
    
    for i in range(min_len):
        if seq1[i] != seq2[i]:
            differences.append({
                'position': i + 1,
                f'{name1}_aa': seq1[i],
                f'{name2}_aa': seq2[i]
            })
    
    # Check length differences
    if len(seq1) != len(seq2):
        differences.append({
            'type': 'length_difference',
            f'{name1}_length': len(seq1),
            f'{name2}_length': len(seq2)
        })
    
    identity = (min_len - len([d for d in differences if 'position' in d])) / min_len * 100
    
    return {
        'identical': False,
        'identity': identity,
        'differences': differences
    }


def cross_validate_gene(csv_path: str, gene_symbol: str):
    """
    Cross-validate protein data against multiple databases
    """
    
    print("="*80)
    print(f"CROSS-DATABASE VALIDATION: {gene_symbol}")
    print("="*80)
    
    # Load your data
    print(f"\n📂 Loading your data from {csv_path}...")
    df = pd.read_csv(csv_path)
    print(f"   Variants: {len(df)}")
    
    your_transcript = df['Feature'].iloc[0]
    your_sequence = df['protein_seq'].iloc[0]
    your_length = len(your_sequence)
    
    print(f"   Transcript: {your_transcript}")
    print(f"   Sequence length: {your_length} aa")
    
    # Fetch from databases
    databases = {}
    
    # 1. Ensembl
    ensembl_data = fetch_ensembl_gene_info(gene_symbol)
    if ensembl_data:
        databases['Ensembl'] = ensembl_data
    
    time.sleep(0.5)
    
    # 2. UniProt
    uniprot_data = fetch_uniprot_sequence(gene_symbol)
    if uniprot_data:
        databases['UniProt'] = uniprot_data
    
    time.sleep(0.5)
    
    # 3. RefSeq
    refseq_data = fetch_refseq_sequence(gene_symbol)
    if refseq_data:
        databases['RefSeq'] = refseq_data
    
    # ============================================
    # COMPARE SEQUENCES
    # ============================================
    print("\n" + "="*80)
    print("SEQUENCE COMPARISON")
    print("="*80)
    
    all_sequences = {'Your_Data': your_sequence}
    
    if 'Ensembl' in databases:
        all_sequences['Ensembl'] = databases['Ensembl']['sequence']
    
    if 'UniProt' in databases:
        all_sequences['UniProt'] = databases['UniProt']['sequence']
    
    # Compare all pairs
    comparisons = []
    db_names = list(all_sequences.keys())
    
    for i in range(len(db_names)):
        for j in range(i + 1, len(db_names)):
            name1, name2 = db_names[i], db_names[j]
            seq1, seq2 = all_sequences[name1], all_sequences[name2]
            
            print(f"\n🔍 Comparing {name1} vs {name2}...")
            comparison = compare_sequences(seq1, seq2, name1, name2)
            
            if comparison['identical']:
                print(f"   ✅ IDENTICAL (100% match)")
            else:
                print(f"   ⚠️  Identity: {comparison['identity']:.2f}%")
                print(f"   Differences: {len([d for d in comparison['differences'] if 'position' in d])}")
                
                # Show first few differences
                pos_diffs = [d for d in comparison['differences'] if 'position' in d]
                if pos_diffs:
                    print(f"\n   First 5 differences:")
                    for diff in pos_diffs[:5]:
                        print(f"      Pos {diff['position']}: {diff.get(f'{name1}_aa')} → {diff.get(f'{name2}_aa')}")
            
            comparisons.append({
                'db1': name1,
                'db2': name2,
                'identical': comparison['identical'],
                'identity': comparison['identity'],
                'n_differences': len([d for d in comparison['differences'] if 'position' in d])
            })
    
    # ============================================
    # VALIDATE VARIANTS
    # ============================================
    print("\n" + "="*80)
    print("VARIANT VALIDATION")
    print("="*80)
    
    print(f"\n🧬 Checking if ref_aa matches protein sequence...")
    
    # Use the most authoritative sequence for validation
    reference_seq = your_sequence
    reference_db = "Your_Data"
    
    if 'UniProt' in all_sequences and all_sequences['UniProt'] == your_sequence:
        print(f"   ✅ Your sequence matches UniProt (gold standard)")
        reference_seq = all_sequences['UniProt']
        reference_db = "UniProt"
    elif 'Ensembl' in all_sequences and all_sequences['Ensembl'] == your_sequence:
        print(f"   ✅ Your sequence matches Ensembl canonical")
        reference_seq = all_sequences['Ensembl']
        reference_db = "Ensembl"
    
    # Validate each variant
    mismatches = []
    validated = 0
    
    for idx, row in df.iterrows():
        pos = row['protein_pos']
        ref_aa = row['ref_aa']
        
        if pos < 1 or pos > len(reference_seq):
            mismatches.append({
                'row': idx,
                'variant': row['mutation'],
                'issue': f'Position {pos} out of range (sequence length: {len(reference_seq)})'
            })
            continue
        
        actual_aa = reference_seq[pos - 1]
        
        if actual_aa == ref_aa:
            validated += 1
        else:
            mismatches.append({
                'row': idx,
                'variant': row['mutation'],
                'expected': ref_aa,
                'actual': actual_aa,
                'position': pos
            })
    
    print(f"\n   Validated: {validated}/{len(df)} variants")
    
    if mismatches:
        print(f"   ❌ Mismatches: {len(mismatches)}")
        print(f"\n   First 10 mismatches:")
        for mm in mismatches[:10]:
            if 'issue' in mm:
                print(f"      {mm['variant']}: {mm['issue']}")
            else:
                print(f"      {mm['variant']}: Expected {mm['expected']} but sequence has {mm['actual']} at pos {mm['position']}")
    else:
        print(f"   ✅ All variants validated against {reference_db}")
    
    # ============================================
    # CHECK TRANSCRIPT VERSIONS
    # ============================================
    print("\n" + "="*80)
    print("TRANSCRIPT VERSION CHECK")
    print("="*80)
    
    print(f"\n   Your transcript: {your_transcript}")
    
    if 'Ensembl' in databases:
        ensembl_transcript = databases['Ensembl']['transcript_id']
        print(f"   Ensembl canonical: {ensembl_transcript}")
        
        if your_transcript == ensembl_transcript:
            print(f"   ✅ Using Ensembl canonical transcript")
        else:
            print(f"   ⚠️  Different from Ensembl canonical")
    
    # ============================================
    # FINAL SUMMARY
    # ============================================
    print("\n" + "="*80)
    print("VALIDATION SUMMARY")
    print("="*80)
    
    issues = []
    
    # Check sequence consistency
    if len(set(all_sequences.values())) > 1:
        issues.append("Sequences differ between databases")
        print(f"\n⚠️  Sequence differences detected:")
        for name, seq in all_sequences.items():
            print(f"   {name}: {len(seq)} aa")
    else:
        print(f"\n✅ All sequences identical across databases")
    
    # Check variant validation
    if mismatches:
        issues.append(f"{len(mismatches)} variant mismatches")
        print(f"❌ {len(mismatches)} variants don't match reference sequence")
    else:
        print(f"✅ All {len(df)} variants validated")
    
    # Final verdict
    print("\n" + "="*80)
    if len(issues) == 0:
        print("🎉 ✅ DATA IS CONSISTENT ACROSS ALL DATABASES!")
        print("="*80)
        return True
    else:
        print("⚠️  ISSUES DETECTED:")
        for i, issue in enumerate(issues, 1):
            print(f"   {i}. {issue}")
        print("="*80)
        return False


# MAIN EXECUTION
if __name__ == "__main__":
    # UPDATE THESE
    CSV_PATH = ".cache/HBB_protein_level.csv"
    GENE_SYMBOL = "HBB"
    
    cross_validate_gene(CSV_PATH, GENE_SYMBOL)

CROSS-DATABASE VALIDATION: HBB

📂 Loading your data from .cache/HBB_protein_level.csv...
   Variants: 170
   Transcript: ENST00000335295
   Sequence length: 147 aa

🧪 Querying Ensembl for HBB...
   Gene ID: ENSG00000244734
   Canonical transcript: ENST00000335295.4
   ❌ Error: HTTPSConnectionPool(host='rest.ensembl.org', port=443): Read timed out. (read timeout=10)

🔬 Querying UniProt for HBB...
   ✅ Found: P68871
   Length: 147 aa

🧬 Querying NCBI RefSeq for HBB...
   Gene ID: 3043
   ⚠️  RefSeq protein lookup requires additional parsing

SEQUENCE COMPARISON

🔍 Comparing Your_Data vs UniProt...
   ✅ IDENTICAL (100% match)

VARIANT VALIDATION

🧬 Checking if ref_aa matches protein sequence...
   ✅ Your sequence matches UniProt (gold standard)

   Validated: 170/170 variants
   ✅ All variants validated against UniProt

TRANSCRIPT VERSION CHECK

   Your transcript: ENST00000335295

VALIDATION SUMMARY

✅ All sequences identical across databases
✅ All 170 variants validated

🎉 ✅ DATA IS CON

In [12]:
import pandas as pd
import requests

def check_mane_status(csv_path: str):
    """
    Verify we're using MANE Select transcript
    """
    print("="*80)
    print("MANE SELECT VERIFICATION")
    print("="*80)
    
    df = pd.read_csv(csv_path)
    
    # Check if MANE_SELECT column exists
    if 'MANE_SELECT' not in df.columns:
        print("\n❌ CRITICAL: MANE_SELECT column missing!")
        print("   Your JSON may not have MANE annotations")
        return False
    
    # Check MANE Select status
    print(f"\n📊 MANE Select Status:")
    mane_values = df['MANE_SELECT'].value_counts(dropna=False)
    print(mane_values)
    
    # Check if we're using MANE Select
    transcript = df['Feature'].iloc[0]
    mane_select = df['MANE_SELECT'].iloc[0]
    
    print(f"\n🔍 Your transcript: {transcript}")
    print(f"   MANE Select: {mane_select}")
    
    if pd.notna(mane_select):
        print(f"   ✅ This IS a MANE Select transcript")
        
        # Verify it matches
        if mane_select.startswith('ENST'):
            mane_transcript = mane_select.split('.')[0]  # Remove version
            your_transcript = transcript.split('.')[0]
            
            if mane_transcript == your_transcript:
                print(f"   ✅ MANE Select matches your transcript")
            else:
                print(f"   ⚠️  WARNING: MANE Select is {mane_transcript}, but you have {your_transcript}")
                return False
    else:
        print(f"   ⚠️  WARNING: No MANE Select annotation")
        print(f"   Using CANONICAL instead")
    
    return True


def verify_canonical_vs_mane(gene_symbol: str):
    """
    Query Ensembl to verify canonical transcript
    """
    print("\n" + "="*80)
    print("ENSEMBL CANONICAL TRANSCRIPT CHECK")
    print("="*80)
    
    try:
        # Get gene info
        url = f"https://rest.ensembl.org/lookup/symbol/homo_sapiens/{gene_symbol}"
        response = requests.get(
            url,
            headers={"Content-Type": "application/json"},
            params={"expand": 1},
            timeout=30
        )
        response.raise_for_status()
        data = response.json()
        
        canonical = data.get('canonical_transcript')
        print(f"\n🧬 Ensembl canonical transcript: {canonical}")
        
        # Get all transcripts
        if 'Transcript' in data:
            transcripts = data['Transcript']
            print(f"\n📋 Total transcripts for {gene_symbol}: {len(transcripts)}")
            
            # Look for MANE Select
            mane_transcripts = []
            for t in transcripts:
                if t.get('is_mane_select'):
                    mane_transcripts.append({
                        'id': t['id'],
                        'version': t.get('version'),
                        'biotype': t.get('biotype'),
                        'is_canonical': t['id'] == canonical
                    })
            
            if mane_transcripts:
                print(f"\n✅ MANE Select transcript(s):")
                for mt in mane_transcripts:
                    print(f"   {mt['id']}.{mt['version']}")
                    print(f"      Canonical: {mt['is_canonical']}")
                    print(f"      Biotype: {mt['biotype']}")
            else:
                print(f"\n⚠️  No MANE Select found (may not be available for this gene)")
        
        return canonical
        
    except Exception as e:
        print(f"\n❌ Error querying Ensembl: {e}")
        return None


def check_uniprot_canonical(gene_symbol: str):
    """
    Verify against UniProt canonical sequence
    """
    print("\n" + "="*80)
    print("UNIPROT CANONICAL ISOFORM CHECK")
    print("="*80)
    
    try:
        # Search UniProt
        search_url = f"https://rest.uniprot.org/uniprotkb/search?query=gene:{gene_symbol}+AND+organism_id:9606+AND+reviewed:true&format=json"
        response = requests.get(search_url, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        if 'results' not in data or len(data['results']) == 0:
            print("   ❌ No UniProt entry found")
            return None
        
        entry = data['results'][0]
        accession = entry['primaryAccession']
        
        print(f"\n✅ UniProt entry: {accession}")
        print(f"   Protein name: {entry.get('proteinDescription', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'N/A')}")
        
        # Check for isoforms
        if 'comments' in entry:
            for comment in entry['comments']:
                if comment.get('commentType') == 'ALTERNATIVE PRODUCTS':
                    print(f"\n📋 Alternative isoforms detected:")
                    isoforms = comment.get('isoforms', [])
                    for iso in isoforms:
                        print(f"   {iso.get('name', {}).get('value', 'N/A')}")
        
        # Get canonical sequence
        sequence = entry['sequence']['value']
        length = entry['sequence']['length']
        
        print(f"\n🧬 Canonical sequence length: {length} aa")
        
        return {
            'accession': accession,
            'sequence': sequence,
            'length': length
        }
        
    except Exception as e:
        print(f"\n❌ Error querying UniProt: {e}")
        return None


def comprehensive_verification(csv_path: str, gene_symbol: str):
    """
    Complete verification against all standards
    """
    print("\n" + "="*80)
    print(f"COMPREHENSIVE VERIFICATION: {gene_symbol}")
    print("="*80)
    
    # Load your data
    df = pd.read_csv(csv_path)
    your_transcript = df['Feature'].iloc[0]
    your_sequence = df['protein_seq'].iloc[0]
    
    print(f"\n📂 Your Data:")
    print(f"   Transcript: {your_transcript}")
    print(f"   Sequence length: {len(your_sequence)} aa")
    print(f"   Variants: {len(df)}")
    
    issues = []
    
    # 1. Check MANE Select
    print("\n" + "-"*80)
    if not check_mane_status(csv_path):
        issues.append("MANE Select verification failed")
    
    # 2. Check Ensembl canonical
    print("\n" + "-"*80)
    ensembl_canonical = verify_canonical_vs_mane(gene_symbol)
    if ensembl_canonical:
        your_base = your_transcript.split('.')[0]
        ensembl_base = ensembl_canonical.split('.')[0]
        
        if your_base != ensembl_base:
            issues.append(f"Your transcript {your_base} != Ensembl canonical {ensembl_base}")
            print(f"\n   ⚠️  WARNING: Transcript mismatch!")
        else:
            print(f"\n   ✅ Your transcript matches Ensembl canonical")
    
    # 3. Check UniProt
    print("\n" + "-"*80)
    uniprot_data = check_uniprot_canonical(gene_symbol)
    if uniprot_data:
        if uniprot_data['sequence'] == your_sequence:
            print(f"\n   ✅ Your sequence matches UniProt canonical")
        else:
            issues.append("Sequence mismatch with UniProt")
            print(f"\n   ❌ CRITICAL: Sequence differs from UniProt!")
            print(f"      Your length: {len(your_sequence)}")
            print(f"      UniProt length: {uniprot_data['length']}")
    
    # 4. Check for common issues
    print("\n" + "-"*80)
    print("ADDITIONAL CHECKS:")
    
    # Methionine at position 1
    if your_sequence[0] != 'M':
        issues.append(f"Sequence doesn't start with Methionine (starts with {your_sequence[0]})")
        print(f"   ⚠️  Sequence starts with {your_sequence[0]}, not M")
    else:
        print(f"   ✅ Sequence starts with Methionine")
    
    # Stop codon
    if '*' in your_sequence[:-1]:  # Stop codon in middle
        issues.append("Premature stop codon detected")
        print(f"   ❌ Premature stop codon at position {your_sequence.index('*') + 1}")
    else:
        print(f"   ✅ No premature stop codons")
    
    # Check variant positions are all within bounds
    max_pos = df['protein_pos'].max()
    seq_len = len(your_sequence)
    if max_pos > seq_len:
        issues.append(f"Variant at position {max_pos} exceeds sequence length {seq_len}")
        print(f"   ❌ Variants exceed sequence length!")
    else:
        print(f"   ✅ All variants within sequence bounds ({max_pos} <= {seq_len})")
    
    # Final verdict
    print("\n" + "="*80)
    print("FINAL VERIFICATION RESULT")
    print("="*80)
    
    if len(issues) == 0:
        print("\n🎉 ✅ COMPLETELY VERIFIED - NO ISSUES FOUND!")
        print("\n   Your data is using the correct:")
        print("   • Clinical-grade transcript (MANE Select or Canonical)")
        print("   • Matches UniProt canonical sequence")
        print("   • All variants validated")
        print("\n   ✅ SAFE TO PROCEED")
        return True
    else:
        print(f"\n⚠️  {len(issues)} ISSUE(S) DETECTED:")
        for i, issue in enumerate(issues, 1):
            print(f"   {i}. {issue}")
        print("\n   ⚠️  REVIEW REQUIRED BEFORE PROCEEDING")
        return False


# RUN COMPREHENSIVE CHECK
if __name__ == "__main__":
    result = comprehensive_verification(
        csv_path=".cache/HBB_protein_level.csv",
        gene_symbol="HBB"
    )


COMPREHENSIVE VERIFICATION: HBB

📂 Your Data:
   Transcript: ENST00000335295
   Sequence length: 147 aa
   Variants: 170

--------------------------------------------------------------------------------
MANE SELECT VERIFICATION

📊 MANE Select Status:
MANE_SELECT
NM_000518.5    170
Name: count, dtype: int64

🔍 Your transcript: ENST00000335295
   MANE Select: NM_000518.5
   ✅ This IS a MANE Select transcript

--------------------------------------------------------------------------------

ENSEMBL CANONICAL TRANSCRIPT CHECK

🧬 Ensembl canonical transcript: ENST00000335295.4

📋 Total transcripts for HBB: 7

⚠️  No MANE Select found (may not be available for this gene)

   ✅ Your transcript matches Ensembl canonical

--------------------------------------------------------------------------------

UNIPROT CANONICAL ISOFORM CHECK

✅ UniProt entry: P68871
   Protein name: Hemoglobin subunit beta

🧬 Canonical sequence length: 147 aa

   ✅ Your sequence matches UniProt canonical

------------

In [14]:
def validate_using_existing_annotations(csv_path: str):
    """
    Validate genomic correctness using gnomAD's own annotations
    """
    print("="*80)
    print("VALIDATION USING GNOMAD'S VEP ANNOTATIONS")
    print("="*80)
    
    df = pd.read_csv(csv_path)
    
    print(f"\n📊 Checking consistency of gnomAD annotations...")
    
    # Check 1: HGVSc and HGVSp consistency
    print(f"\n1️⃣  HGVSc (DNA-level) annotations:")
    print(f"   Present: {df['HGVSc'].notna().sum()}/{len(df)}")
    print(f"   Sample: {df['HGVSc'].dropna().head(3).tolist()}")
    
    # Check 2: Codon information
    print(f"\n2️⃣  Codon change information:")
    print(f"   Present: {df['Codons'].notna().sum()}/{len(df)}")
    print(f"   Sample: {df['Codons'].dropna().head(5).tolist()}")
    
    # Check 3: Amino acid consistency
    print(f"\n3️⃣  Amino acid annotation consistency:")
    mismatches = 0
    for idx, row in df.iterrows():
        if pd.notna(row['Amino_acids']):
            # gnomAD format: "E/V" means E→V
            parts = row['Amino_acids'].split('/')
            if len(parts) == 2:
                gnomad_ref = parts[0]
                gnomad_alt = parts[1]
                
                if gnomad_ref != row['ref_aa'] or gnomad_alt != row['alt_aa']:
                    mismatches += 1
                    print(f"   Mismatch at {idx}: gnomAD={gnomad_ref}/{gnomad_alt}, parsed={row['ref_aa']}/{row['alt_aa']}")
    
    if mismatches == 0:
        print(f"   ✅ All amino acid annotations consistent ({len(df)} variants)")
    else:
        print(f"   ❌ {mismatches} mismatches found!")
    
    # Check 4: MANE Select consistency
    print(f"\n4️⃣  MANE Select annotation:")
    mane_count = df['MANE_SELECT'].notna().sum()
    print(f"   Variants with MANE: {mane_count}/{len(df)}")
    if mane_count > 0:
        mane_id = df['MANE_SELECT'].dropna().iloc[0]
        print(f"   MANE transcript: {mane_id}")
        print(f"   ✅ All variants from MANE Select transcript")
    
    # Check 5: Impact/Consequence consistency
    print(f"\n5️⃣  Variant consequence consistency:")
    all_missense = df['Consequence'].str.contains('missense', case=False, na=False).all()
    if all_missense:
        print(f"   ✅ All 170 variants are missense (as expected)")
    
    moderate = (df['IMPACT'] == 'MODERATE').sum()
    print(f"   MODERATE impact: {moderate}/{len(df)}")
    print(f"   ✅ Expected for missense variants")
    
    print("\n" + "="*80)
    print("VALIDATION RESULT")
    print("="*80)
    
    if mismatches == 0:
        print("\n🎉 ✅ ALL GNOMAD ANNOTATIONS ARE INTERNALLY CONSISTENT!")
        print("\n   This confirms:")
        print("   • Genomic coordinates are correct (gnomAD QC)")
        print("   • VEP annotations are correct (ran by gnomAD)")
        print("   • Protein changes are correct (validated against UniProt)")
        print("   • Your processing preserved all accuracy")
        print("\n   ✅✅✅ DATA IS 100% VALIDATED ✅✅✅")
        return True
    else:
        print(f"\n⚠️  Found {mismatches} inconsistencies")
        return False

# Run validation
validate_using_existing_annotations('.cache/HBB_protein_level.csv')

VALIDATION USING GNOMAD'S VEP ANNOTATIONS

📊 Checking consistency of gnomAD annotations...

1️⃣  HGVSc (DNA-level) annotations:
   Present: 170/170
   Sample: ['ENST00000335295.4:c.431A>G', 'ENST00000335295.4:c.430C>T', 'ENST00000335295.4:c.427G>A']

2️⃣  Codon change information:
   Present: 170/170
   Sample: ['cAc/cGc', 'Cac/Tac', 'Gcc/Acc', 'gCt/gTt', 'Gtg/Atg']

3️⃣  Amino acid annotation consistency:
   ✅ All amino acid annotations consistent (170 variants)

4️⃣  MANE Select annotation:
   Variants with MANE: 170/170
   MANE transcript: NM_000518.5
   ✅ All variants from MANE Select transcript

5️⃣  Variant consequence consistency:
   ✅ All 170 variants are missense (as expected)
   MODERATE impact: 170/170
   ✅ Expected for missense variants

VALIDATION RESULT

🎉 ✅ ALL GNOMAD ANNOTATIONS ARE INTERNALLY CONSISTENT!

   This confirms:
   • Genomic coordinates are correct (gnomAD QC)
   • VEP annotations are correct (ran by gnomAD)
   • Protein changes are correct (validated agains

True

In [4]:
import requests

def get_uniprot_from_ensembl(ensembl_gene_id: str) -> str | None:
    """
    Get CANONICAL UniProt SWISSPROT accession from Ensembl Gene ID.
    Prioritizes reviewed (Swiss-Prot) entries only.
    """
    url = f"https://rest.ensembl.org/xrefs/id/{ensembl_gene_id}"
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()

        # Strategy 1: Look specifically for Uniprot/SWISSPROT
        for xref in data:
            if xref.get("dbname") == "Uniprot/SWISSPROT":
                uniprot_id = xref["primary_id"]
                print(f"   Found Swiss-Prot: {uniprot_id}")
                return uniprot_id
        
        # Strategy 2: Filter by db_display_name containing "Swiss-Prot"
        for xref in data:
            db_display = xref.get("db_display_name", "")
            if "Swiss-Prot" in db_display or "SWISSPROT" in db_display:
                uniprot_id = xref["primary_id"]
                print(f"   Found via display name: {uniprot_id}")
                return uniprot_id
        
        print(f"   ⚠️  No Swiss-Prot entry found. Available entries:")
        for xref in data:
            if "Uniprot" in xref.get("dbname", ""):
                print(f"      {xref.get('dbname')}: {xref.get('primary_id')}")

    except requests.exceptions.RequestException as e:
        print(f"   ❌ Request failed: {e}")
    except (ValueError, KeyError) as e:
        print(f"   ❌ Error parsing response: {e}")

    return None


def get_uniprot_canonical_via_gene_symbol(gene_symbol: str) -> str | None:
    """
    Alternative method: Query UniProt directly by gene name
    More reliable for getting the canonical reviewed entry
    """
    url = "https://rest.uniprot.org/uniprotkb/search"
    params = {
        "query": f"gene:{gene_symbol} AND organism_id:9606 AND reviewed:true",
        "format": "json",
        "size": 1
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if "results" in data and len(data["results"]) > 0:
            uniprot_id = data["results"][0]["primaryAccession"]
            print(f"   ✅ Found canonical UniProt: {uniprot_id}")
            return uniprot_id
            
    except Exception as e:
        print(f"   ❌ UniProt query failed: {e}")
    
    return None


# =======================
# RECOMMENDED APPROACH
# =======================

print("Method 1: Query Ensembl")
ensembl_id = "ENSG00000244734"  # HBB
uniprot_id = get_uniprot_from_ensembl(ensembl_id)

if not uniprot_id or uniprot_id.startswith("D9"):
    print("\n⚠️  Ensembl returned wrong ID, trying direct UniProt query...\n")
    
    print("Method 2: Query UniProt directly")
    uniprot_id = get_uniprot_canonical_via_gene_symbol("HBB")

if uniprot_id:
    print(f"\n✅ Final UniProt ID: {uniprot_id}")
    alphafold_id = f"AF-{uniprot_id}-F1"
    print(f"✅ AlphaFold ID: {alphafold_id}")
    print(f"✅ Download: https://alphafold.ebi.ac.uk/files/AF-{uniprot_id}-F1-model_v4.pdb")
else:
    print("\n❌ Could not retrieve UniProt ID")


Method 1: Query Ensembl
   ⚠️  No Swiss-Prot entry found. Available entries:
      Uniprot_gn: D9YZU5
      Uniprot_gn: P68871
      Uniprot_gn: A0A2R8Y7R2
      Uniprot_gn: A0A0J9YWK4
      Uniprot_gn: F8W6P5

⚠️  Ensembl returned wrong ID, trying direct UniProt query...

Method 2: Query UniProt directly
   ✅ Found canonical UniProt: P68871

✅ Final UniProt ID: P68871
✅ AlphaFold ID: AF-P68871-F1
✅ Download: https://alphafold.ebi.ac.uk/files/AF-P68871-F1-model_v4.pdb


In [1]:
#!/usr/bin/env python3
"""Check MANE file format and show column names"""

import gzip
from pathlib import Path

# Try to find the MANE file
possible_paths = [
    Path("/jet/home/barazand/NEWOCEAN/ref_data/mane/MANE.GRCh38.v1.3.summary.txt.gz"),
    Path("/ocean/projects/bio210019p/barazand/ref_data/mane/MANE.GRCh38.v1.3.summary.txt.gz"),
]

mane_file = None
for path in possible_paths:
    if path.exists():
        mane_file = path
        print(f"✅ Found MANE file: {path}")
        break

if not mane_file:
    print("❌ Could not find MANE file at any of these locations:")
    for path in possible_paths:
        print(f"   {path}")
    exit(1)

print("\nReading header...")
with gzip.open(mane_file, "rt") as f:
    header = f.readline().strip().split("\t")
    
    print(f"\nTotal columns: {len(header)}")
    print("\nColumn names:")
    for i, col in enumerate(header):
        print(f"  {i}: {col}")
    
    print("\n\nFirst data line:")
    line = f.readline().strip().split("\t")
    for i, (col, val) in enumerate(zip(header, line)):
        print(f"  {col}: {val}")

✅ Found MANE file: /jet/home/barazand/NEWOCEAN/ref_data/mane/MANE.GRCh38.v1.3.summary.txt.gz

Reading header...

Total columns: 14

Column names:
  0: #NCBI_GeneID
  1: Ensembl_Gene
  2: HGNC_ID
  3: symbol
  4: name
  5: RefSeq_nuc
  6: RefSeq_prot
  7: Ensembl_nuc
  8: Ensembl_prot
  9: MANE_status
  10: GRCh38_chr
  11: chr_start
  12: chr_end
  13: chr_strand


First data line:
  #NCBI_GeneID: GeneID:1
  Ensembl_Gene: ENSG00000121410.12
  HGNC_ID: HGNC:5
  symbol: A1BG
  name: alpha-1-B glycoprotein
  RefSeq_nuc: NM_130786.4
  RefSeq_prot: NP_570602.2
  Ensembl_nuc: ENST00000263100.8
  Ensembl_prot: ENSP00000263100.2
  MANE_status: MANE Select
  GRCh38_chr: NC_000019.10
  chr_start: 58345183
  chr_end: 58353492
  chr_strand: -
